In [7]:
from langchain_community.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


llm = Ollama(
    model = "llama3.1", callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
)
answer = llm.invoke("football winner 2022")


/var/folders/vv/jld3_3hn5fz42wp20lz51yxm0000gn/T/ipykernel_98973/49787312.py:6: DeprecationWarning: callback_manager is deprecated. Please use callbacks instead.
  llm = Ollama(


You're referring to the winners of various football competitions in 2022!

Here are some notable ones:

1. **FIFA World Cup Qatar 2022**:
	* Winners: Argentina (won against France in the final on December 18, 2022)
	* Golden Ball (best player): Lionel Messi
	* Golden Glove (best goalkeeper): Emiliano Martínez
2. **UEFA Champions League 2021-22**:
	* Winners: Real Madrid (beat Liverpool 1-0 in the final on May 28, 2022)
3. **English Premier League 2021-22**:
	* Winners: Manchester City
4. **La Liga 2021-22**:
	* Winners: Barcelona (on points average, as the team finished with the same number of points but fewer goals conceded than Real Madrid)
5. **Bundesliga 2021-22**:
	* Winners: Bayern Munich
6. **Serie A 2021-22**:
	* Winners: AC Milan
7. **UEFA Europa League 2021-22**:
	* Winners: Eintracht Frankfurt (beat Rangers 2-1 in the final on May 18, 2022)
8. **Copa Libertadores 2022**:
	* Winners: Independiente del Valle (from Ecuador)

These are just a few examples of football winners fro

## Text Extraction

In [31]:
from langchain_community.document_loaders import PyPDFLoader

pdf_kappa = "./pdfs/Kappe.pdf"
loader = PyPDFLoader(pdf_kappa)

# Load all pages and extract text content
pages = [page.page_content for page in loader.lazy_load()]

In [33]:
pages

['Recommender Systems and Nudges\nfor Healthier Food Choices\nAyoub El Majjodi\nDissertationforthedegreeofphilosophiaedoctor(PhD)\nattheUniversityofBergen,Norway\n2025\nDissertationdate:',
 '©CopyrightAyoubElMajjodi\nThematerialinthispublicationisprotectedbycopyrightlaw.\nYear: 2025\nTitle: RecommenderSystemsandNudgesforHealthierFoodChoices\nAuthor: AyoubElMajjodi',
 'Scientific environment\nTheresearchpresentedinthisdissertationwasconductedattheResearchCentreforRespon-\nsibleMediaTechnology&Innovation(MediaFutures)andtheBehavioralDataAnalytics&\nRecommenderSystemsResearchGroup, hosted bytheDepartmentofInformationScience\nandMediaStudiesattheUniversityof Bergen,Bergen,Norway. ItwassupervisedbyProf.\nChristophTrattnerandco-supervisedbyAss. Prof. AlainD.Starke.',
 'iv Scientificenvironment',
 'Acknowledgements\nFirst and foremost, I would like to express my deepest gratitude to my\nsupervisors, Prof. Christoph T rattner and Ass. Prof. Alain D. Starke,\nfor their invaluable guidance, cont

In [30]:
pip install pypdf

Note: you may need to restart the kernel to use updated packages.
